In [1]:
# 라이브러리 호출
%matplotlib inline
import glob
import json
from pathlib import Path
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate
from tqdm.auto import tqdm
import shutil
import cv2
import numpy as np
import pickle
LANGUAGE_LIST = ['japanese', 'chinese', 'thai', 'vietnamese']
LANGUAGE_LIST_SUB = {'japanese':'ja','chinese':'zh', 'thai':'th', 'vietnamese':'vi'}

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

def read_json(filename):
    with Path(filename).open(encoding='utf8') as handle:
        ann = json.load(handle)
    return ann

path_lists = glob.glob(f"../data/bottom_iou_data/*_receipt/ufo/train.json")

data = {}
data['images'] = {}
for path in path_lists:    
    json_data = read_json(path)
    images = list(json_data['images'].items())
    data['images'].update(dict(images))

print(f"Total number of GT_images: {len(data['images'])}")

path_lists_100 = glob.glob(f"t2048output.json")
#path_lists_100 = glob.glob(f"train_Re.json")
data_train100 = {}
data_train100['images'] = {}
for path in path_lists_100:
    json_data = read_json(path)
    images = list(json_data['images'].items())
    data_train100['images'].update(dict(images))

print(f"Total number of pred_images: {len(data_train100['images'])}")

Total number of GT_images: 398
Total number of pred_images: 398


In [50]:
keys_list = sorted(list(data['images'].keys()))
ja_keys_list = list()
zh_keys_list = list()
th_keys_list = list()
vi_keys_list = list()

for key in keys_list:
    if key.split('.')[1] == 'ja':
        ja_keys_list.append(key)
    elif key.split('.')[1] == 'zh':
        zh_keys_list.append(key)
    elif key.split('.')[1] == 'th':
        th_keys_list.append(key)
    elif key.split('.')[1] == 'vi':
        vi_keys_list.append(key)
t_key_list = [sorted(ja_keys_list),sorted(zh_keys_list),sorted(th_keys_list),sorted(vi_keys_list)]
for lan in t_key_list:
    print(len(lan))

99
99
100
100


In [52]:
def get_box_angle(box):
    # 박스의 긴 변 방향을 찾습니다
    coords = np.array(box.exterior.coords)
    edges = np.diff(coords, axis=0)
    longest_edge = edges[np.argmax(np.sum(edges**2, axis=1))]
    angle = np.arctan2(longest_edge[1], longest_edge[0])
    return angle

def angle_difference(box1, box2):
    angle1 = get_box_angle(box1)
    angle2 = get_box_angle(box2)
    diff = angle2 - angle1
    # 각도 차이를 -pi에서 pi 사이로 정규화합니다
    return (diff + np.pi) % (2 * np.pi) - np.pi

def rotated_boxes_iou(box1, box2):
    poly1 = Polygon(box1)
    poly2 = Polygon(box2)    
    if not poly1.is_valid:
        poly1 = poly1.buffer(0)
    if not poly2.is_valid:
        poly2 = poly2.buffer(0)    
    inter_area = poly1.intersection(poly2).area    
    union_area = poly1.area + poly2.area - inter_area    
    iou = inter_area / union_area if union_area > 0 else 0    
    return iou

# 데이터 저장
iou_data = {}
def store_data(language, image, box_num, box, box2, iou, angle_diff):
    if language not in iou_data:
        iou_data[language] = {}
    if image not in iou_data[language]:
        iou_data[language][image] = {}
    iou_data[language][image][box_num] = {'pbox':box, 'ybox':box2, 'iou': iou, 'angle_diff': angle_diff}

def get_data(language, image, box_num):
    return iou_data[language][image][box_num]

# 모든 나라
for lang in t_key_list:
    country = lang[0].split('.')[1]
    # 나라별 이미지
    for li in tqdm(lang):
        # 이미지별 박스
        for p1 in data['images'][li]['words']:
            max = 0 
            mp2 = None
            box1 = data['images'][li]['words'][p1]['points']
            for p2 in data_train100['images'][li]['words']:
                box2 = data_train100['images'][li]['words'][p2]['points']
                iou = rotated_boxes_iou(box1, box2)
                if max < iou:
                    max = iou
                    mp2 = p2
            angle_diff = None
            if not mp2 == None: 
                box2 = data_train100['images'][li]['words'][mp2]['points']
                angle_diff = angle_difference(Polygon(box1), Polygon(box2))
            store_data(country,li,p1,box1,box2,max,angle_diff)

100%|██████████| 100/100 [00:37<00:00,  2.69it/s]


In [5]:
iou_datas = iou_data 
iou_datas['ja']

{'extractor.ja.in_house.appen_000028_page0001.jpg': {'1': {'pbox': [[220.76,
     1483.0],
    [355.87, 1486.6],
    [354.25, 1547.55],
    [219.13, 1543.94]],
   'ybox': [[209.7201690673828, 1482.56787109375],
    [354.2370910644531, 1484.1124267578125],
    [353.54052734375, 1549.3153076171875],
    [209.02362060546875, 1547.770263671875]],
   'iou': 0.8602704566038619,
   'angle_diff': 3.1255693234174657},
  '2': {'pbox': [[384.92, 1484.18],
    [1119.54, 1475.41],
    [1120.31, 1539.96],
    [385.69, 1548.72]],
   'ybox': [[326.6094055175781, 1490.4439697265625],
    [1080.13818359375, 1462.1014404296875],
    [1083.0982666015625, 1540.6361083984375],
    [329.5691223144531, 1568.9801025390625]],
   'iou': 0.7195736402089299,
   'angle_diff': 3.115932962810648},
  '3': {'pbox': [[122.45, 1627.59],
    [447.44, 1627.59],
    [447.44, 1689.68],
    [122.45, 1689.68]],
   'ybox': [[120.95125579833984, 1628.820556640625],
    [434.15484619140625, 1622.221435546875],
    [435.6507873535

In [6]:
def save_iou_data(filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(iou_data, f, ensure_ascii=False, indent=4)

def load_iou_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        iou_data = json.load(f)
    return iou_data
iou_data_filename = 'iou_data.json'
save_iou_data(iou_data_filename)



In [7]:
iou_data = load_iou_data(iou_data_filename)

In [60]:
# IOU 값을 추출하는 함수
def get_all_iou_values():
    iou_values = []
    # 모든 나라와 이미지, 박스를 순회하여 IOU 값을 추출합니다
    for lang in iou_data:  # data는 'data' 딕셔너리
        for image in iou_data[lang]:  # 이미지별
            for box in iou_data[lang][image]:  # 각 박스별                 
                 iou_values.append(iou_data[lang][image][box]['iou'])  # IOU 값 추출
    return iou_values

# IOU 값을 정렬하고 원하는 퍼센트 범위의 값 출력
def get_percentile_iou(percentile):
    iou_values = get_all_iou_values()  # IOU 값 추출
    iou_values.sort()  # IOU 값 오름차순 정렬
    
    # 특정 퍼센트의 값을 계산 (예: 20%, 80% 등)
    index = int(len(iou_values) * percentile / 100)
    percentile_value = iou_values[index]  # 해당 퍼센트의 IOU 값
    
    return percentile_value

# 예시: 상위 20% IOU 값을 출력
top_20_percent_iou = get_percentile_iou(99)  # 80%는 상위 20%
print(f"상위 20% IOU 값: {top_20_percent_iou}")

# 예시: 하위 20% IOU 값을 출력
bottom_30_percent_iou = get_percentile_iou(72)  # 20%는 하위 20%
print(f"하위 30% IOU 값: {bottom_30_percent_iou}")

상위 20% IOU 값: 0.8535507967771531
하위 30% IOU 값: 0.619243096162794


In [41]:
import os
# 이미지에서 기준보다 높은 IOU 값을 가진 박스를 지우고 새로 저장하는 함수
def update_image_with_filtered_boxes(img_path, boxes_to_keep, save_path,blur_strength=51):
    img = cv2.imread(str(img_path))  # 이미지 읽기
    if img is None:
        print(f"이미지를 읽을 수 없습니다: {img_path}")
        return False  
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    height, width = gray_img.shape
    adjusted_blur_strength = np.max([3, int(blur_strength * (np.min([height, width]) / 1000))])
    if adjusted_blur_strength % 2 == 0:
        adjusted_blur_strength += 1

    blurred_img = cv2.GaussianBlur(gray_img, (adjusted_blur_strength, adjusted_blur_strength), 0)
    img_result = gray_img.copy() 
 
    for box in boxes_to_keep:
        pts = np.array(box, dtype=np.int32)
        mask = np.zeros_like(gray_img, dtype=np.uint8)
        cv2.fillPoly(mask, [pts], (255))
        img_result = cv2.bitwise_and(img_result, cv2.bitwise_not(mask))  
        blurred_no_box = cv2.bitwise_and(blurred_img, mask)       
        img_result = cv2.bitwise_or(img_result, blurred_no_box)

    cv2.imwrite(str(save_path), img_result)
    return True

# JSON 파일에서 박스 정보 업데이트
def update_json_with_filtered_boxes(json_path, boxes_to_keep,receipt_dir):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    keys_to_delete = [] 
    for key, image_data in data.get('images', {}).items():
        new_words = {} 
        for word_id, word in image_data.get('words').items():            
            if word_id in boxes_to_keep:  # 'boxes_to_keep'에 포함된 박스만 유지                
                new_words[word_id]=word
        if len(new_words) <= 0 :
            image_file_path = os.path.join(receipt_dir, 'img/train',f"{key}")
            if os.path.exists(image_file_path):
                os.remove(image_file_path)  # 이미지 파일 삭제
                print(f"이미지 파일 삭제됨: {image_file_path}")
            else:
                print(f"이미지 파일이 존재하지 않음: {image_file_path}")
                keys_to_delete.append(key) 
        else:
            image_data['words'] = new_words 
    for key in keys_to_delete:
        if key in data['images']:
            del data['images'][key]
            print(f"이미지 {key} 삭제됨")  # 실제로 삭제된 이미지 확인  
    # 변경된 데이터를 다시 저장
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 하위 30% IOU 값보다 높은 박스를 지우고 새로운 이미지로 저장하는 함수
def filter_and_remove_high_iou_boxes(language, data, bottom_30_iou_value, receipt_dir, bottom_iou_receipt_dir):
    for subset in ['train', 'test']:
        img_dir = receipt_dir / 'img' / subset
        bottom_iou_img_dir = bottom_iou_receipt_dir / 'img' / subset
        boxes_to_keep_id_all = []  # 필터링된 박스를 저장할 리스트
        for img_path in img_dir.glob('*'):            
            # 언어별로 이미지를 처리
            boxes_to_keep_id = []  # 필터링된 박스를 저장할 리스트
            boxes_to_keep_box = [] 
            if subset == 'train':
                image = data[LANGUAGE_LIST_SUB[language]].get(str(img_path).split('/')[-1])
                for box in image:                  
                    d = image[box]
                    iou = d['iou']
                    if iou > bottom_30_iou_value:  # IOU 값이 기준보다 낮으면 박스를 유지
                        boxes_to_keep_box.append(d['pbox'])  # 박스의 좌표를 추가
                    else:
                        boxes_to_keep_id.append(box)
            boxes_to_keep_id_all+=boxes_to_keep_id
            # # `train` 데이터만 필터링하여 저장, `test` 데이터는 복사만
            # if subset == 'train':
            #     bottom_iou_path = bottom_iou_img_dir / f"{img_path.name}"
            #     #print(img_path.name)
            #     if update_image_with_filtered_boxes(img_path, boxes_to_keep_box, bottom_iou_path):
            #         pass
            #         #print(f"Filtered image saved at: {bottom_iou_path}")
            #        # `test` 데이터는 그냥 복사만
            # if subset == 'test':
            #     bottom_iou_path = bottom_iou_img_dir / f"{img_path.name}"
            #     shutil.copy(img_path, bottom_iou_path)  # 이미지 파일을 복사만 진행
            #     #print(f"Copied image at: {bottom_iou_path}")

        print("total len: ",len(boxes_to_keep_id_all))
        if subset == 'train':        
            json_path = bottom_iou_receipt_dir / 'ufo' / 'train.json'  # 모든 이미지가 포함된 train.json을 처리
            update_json_with_filtered_boxes(json_path, boxes_to_keep_id_all,bottom_iou_receipt_dir)
        
 

# 기본 디렉토리 경로 설정
base_dir = Path("../data/re_data")
bottom_iou_base_dir = Path("../data") / "bottom_iou_data"  # 'bottom_iou_data'로 변경

# 언어별로 이미지에서 필터링된 박스를 지우고 새로운 이미지로 저장
for language in LANGUAGE_LIST:
    receipt_dir = base_dir / f"{language}_receipt"
    bottom_iou_receipt_dir = bottom_iou_base_dir / f"{language}_receipt"  # '_bottom_iou' 추가

    for subset in ['train', 'test']:
        (bottom_iou_receipt_dir / 'img' / subset).mkdir(parents=True, exist_ok=True)
    
    shutil.copytree(receipt_dir / 'ufo', bottom_iou_receipt_dir / 'ufo', dirs_exist_ok=True)
    
    # 기준 IOU 값보다 높은 박스를 지운 후, 새로운 이미지로 저장
    filter_and_remove_high_iou_boxes(language, iou_data, bottom_30_percent_iou, receipt_dir, bottom_iou_receipt_dir)
    print(f"{language} 처리가 완료되었습니다.")
    
print("모든 처리가 완료되었습니다.")

PosixPath('../data/bottom_iou_data/japanese_receipt/ufo')

total len:  1992
이미지 파일이 존재하지 않음: ../data/bottom_iou_data/japanese_receipt/img/train/extractor.ja.in_house.appen_000464_page0001.jpg
이미지 extractor.ja.in_house.appen_000464_page0001.jpg 삭제됨
total len:  0
japanese 처리가 완료되었습니다.


PosixPath('../data/bottom_iou_data/chinese_receipt/ufo')

total len:  1931
이미지 파일이 존재하지 않음: ../data/bottom_iou_data/chinese_receipt/img/train/extractor.zh.in_house.appen_000695_page0001.jpg
이미지 extractor.zh.in_house.appen_000695_page0001.jpg 삭제됨
total len:  0
chinese 처리가 완료되었습니다.


PosixPath('../data/bottom_iou_data/thai_receipt/ufo')

total len:  2610
total len:  0
thai 처리가 완료되었습니다.


PosixPath('../data/bottom_iou_data/vietnamese_receipt/ufo')

total len:  2830
total len:  0
vietnamese 처리가 완료되었습니다.
모든 처리가 완료되었습니다.


In [ ]:
def update_image_with_filtered_boxes(img_path, boxes_to_keep, save_path,blur_strength=51):
    img = cv2.imread(str(img_path))  # 이미지 읽기
    if img is None:
        print(f"이미지를 읽을 수 없습니다: {img_path}")
        return False

    # 이미지의 해상도에 따라 블러 강도 동적으로 조정
    height, width = img.shape[:2]
    adjusted_blur_strength = np.max([3, int(blur_strength * (np.min([height, width]) / 1000))])
    if adjusted_blur_strength % 2 == 0:
        adjusted_blur_strength += 1

    
    blurred_img = cv2.GaussianBlur(gray_img, (blur_strength, blur_strength), 0)
    # 필터링된 박스들만 그리기 (기준 이하의 IOU 값만)

    for box in boxes_to_keep:
        pts = np.array(box, dtype=np.int32)

        # 4. 마스크 생성: 동일한 크기의 제로 이미지를 만든 후, 사각형 영역을 흰색으로 채움
        mask = np.zeros_like(img, dtype=np.uint8)

        # 5. 폴리곤 영역을 흰색으로 채우기 (기울어진 사각형)
        cv2.fillPoly(mask, [pts], (255))

        # 6. 다각형 영역을 원본 이미지에서 추출
        img_no_box = cv2.bitwise_and(img, cv2.bitwise_not(mask))  # 원본 이미지에서 다각형 영역을 제외
        blurred_no_box = cv2.bitwise_and(blurred_img, mask)  # 블러 처리된 이미지에서 다각형 영역만 추출

        # 7. 원본 이미지에서 다각형 부분을 블러 처리된 이미지로 덮어쓰기
        img_no_box = cv2.bitwise_or(img_no_box, blurred_no_box)

        # 업데이트된 원본 이미지로 덮어쓰기
        img = img_no_box